In [67]:
from openai import OpenAI
import base64
import requests
import pandas as pd
import io
client = OpenAI()
api_key = os.getenv('OPENAI_API_KEY')

# An Example Page

A demonstration of parsing of one page. 

In [18]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../../data/exhibition_data_pictures/Paris1867_Italy_0.jpg"

In [26]:
# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

In [82]:
payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Parse this image to .csv with \t as a delimiter and the variables Number, Name, Place, Description."
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 1500
}

In [83]:
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9PBPrEpruIF3H8Q8Ob8y6pbGogEyy', 'object': 'chat.completion', 'created': 1715789111, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here is the parsed .csv data with tab delimiters for the provided image:\n\n```\nNumber\tName\tPlace\tDescription\n1\tDalmazzo (Henry)\tTurin\tImprimerie royale, à Turin. — Dictionnaire polyglotte en onze langues, de M. Louis Calligaris.\n2\tNegro (Auguste-Frédéric)\tTurin\tOuvrages imprimés; spécimens de typographie.\n3\tPomba (Louis)\tTurin\tPublications de la Société de l'union typographique de Turin.\n4\tGatti (Joseph)\tVoghera\tSpécimens de typographie.\n5\tBernardoni (Cière-Joseph)\tMilan\tEvangelarium, Epistolarium et Lectionarium, etc. Documents diplomatiques tirés des archives milanaises; Histoire politique, civile et militaire de la maison royale de Savoie; Statistique générale de la province de Sondrio; Supplément aux vocabulaires italiens de Gherardini.\n6\tVallardi (François)

In [84]:
csv_content = response.json()['choices'][0]['message']['content']

# Remove markdown and extract CSV string
csv_string = csv_content.split('```\n')[1].split('```')[0]

In [93]:
df = pd.read_csv(io.StringIO(csv_string), delimiter='\t')

In [96]:
df

,Number,Name,Place,Description
0,1,Dalmazzo (Henry),Turin,"Imprimerie royale, à Turin. — Dictionnaire pol..."
1,2,Negro (Auguste-Frédéric),Turin,Ouvrages imprimés; spécimens de typographie.
2,3,Pomba (Louis),Turin,Publications de la Société de l'union typograp...
3,4,Gatti (Joseph),Voghera,Spécimens de typographie.
4,5,Bernardoni (Cière-Joseph),Milan,"Evangelarium, Epistolarium et Lectionarium, et..."
5,6,Vallardi (François),Milan,Bibliothèque médicale; Encyclopédie nationale;...
6,7,Civelli (Joseph),Milan,Ouvrages d'architecture; grands concours; la C...
7,8,"Trieste (Maso), sous-commission de Padoue",Padoue,Description des chapelles des Scrovegni et de ...
8,9,Antonelli (Joseph),Venise,Ouvrages imprimés et reliés.
9,10,Leocantelli (Lomi et Piucco),Venise,Collection des étrennes vénitiennes.


# A function

A function to loop through all files and export jsons of raw outputs and ready-made csv's per page. 